Este notebook es usado para calcular el valor de los outputs a manera de encontrar el valor esperado para el proceso de testing

In [1]:
import tensorflow as tf

In [5]:
T = tf.constant([[2.0, 3.0], [5.0, 7.0]])
P = tf.constant([[2.5, 3.5], [4.0, 6.0]])

## WMAPE

In [6]:
def WMAPE(T: tf.Tensor, P: tf.Tensor, delta=tf.constant(1e-5, dtype=tf.float32)):
    T = tf.where(T == 0.0, delta, T) # evita la division por 0
    P = tf.where(P == 0.0, delta, P)

    W_fun = lambda T, max_T: 1 + (T / max_T)
    max_T = tf.reduce_max(T)
    W = W_fun(T, max_T)

    Numerator = W * (tf.abs(P - T) / tf.abs(T))
	
    sum_W = tf.reduce_sum(W)
    sum_Numerator = tf.reduce_sum(Numerator)
    return sum_Numerator / sum_W

In [7]:
WMAPE(T, P)

<tf.Tensor: shape=(), dtype=float32, numpy=0.1848148>

## DICE

In [10]:
def DICE_binary_mask(tensor: tf.Tensor):
    tensor_mask = tensor - tf.reduce_mean(tensor)
    tensor_mask = tf.where(tensor_mask > 0, tf.ones_like(tensor_mask), tf.zeros_like(tensor_mask))
    return tensor_mask

def DICEE(T: tf.Tensor, P: tf.Tensor, alpha: float = 0.5, beta: float = 0.5):
    if T.shape != P.shape:
        return None
    G = DICE_binary_mask(T)
    A = DICE_binary_mask(P)
    sum_G_A = tf.reduce_sum(G * A)
    sum_alpha = tf.reduce_sum(A * (1 - G))
    sum_beta = tf.reduce_sum(G * (1 - A))
    return 1 - (sum_G_A / (sum_G_A + alpha * sum_alpha + beta * sum_beta))


In [14]:
tensor = tf.constant([[1.0, 2.0], [-1.0, -2.0]])
DICE_binary_mask(tensor)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [0., 0.]], dtype=float32)>

In [14]:
T = tf.constant([[1.0, -1], [0.0, 0.0]])
P = tf.constant([[1.0, 1.0], [0.0, 0.0]])
DICEE(T, P)

<tf.Tensor: shape=(), dtype=float32, numpy=0.3333333>

<tf.Tensor: shape=(), dtype=float32, numpy=0.3333333>

## DPEAKS

In [16]:
import numpy as np
import tensorflow as tf
from scipy.ndimage import gaussian_filter
from skimage.feature import peak_local_max

def find_top_k_peaks(im, sigma=3, N=3):
    smoothed = gaussian_filter(im, sigma=sigma)
    coordinates = peak_local_max(smoothed, threshold_abs=None, num_peaks=N)
    while len(coordinates) < 3:
        coordinates = np.vstack([coordinates, np.array([0, 0])])
    return coordinates

def DPEAKS(T: np.ndarray, P: np.ndarray, num_peaks=3):
    PEAKS_T = find_top_k_peaks(T, N=num_peaks)
    PEAKS_P = find_top_k_peaks(P, N=num_peaks)
    sum_DPEAKS = np.sum(np.abs(PEAKS_T - PEAKS_P))
    return sum_DPEAKS


In [48]:
import numpy as np

# Create a 10x10 constant tensor with some peaks
center = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 2, 2, 2, 2, 2, 2, 2, 2, 1],
    [1, 2, 3, 3, 3, 3, 3, 3, 2, 1],
    [1, 2, 3, 4, 4, 4, 4, 3, 2, 1],
    [1, 2, 3, 4, 5, 5, 4, 3, 2, 1],
    [1, 2, 3, 4, 5, 5, 4, 3, 2, 1],
    [1, 2, 3, 4, 4, 4, 4, 3, 2, 1],
    [1, 2, 3, 3, 3, 3, 3, 3, 2, 1],
    [1, 2, 2, 2, 2, 2, 2, 2, 2, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
])

# Create a 10x10 tensor with peaks near the borders
border = np.array([
    [0, 1,  1,  1,  1,  1,  1,  1,  8, 10],
    [1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
    [1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
    [1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
    [1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
    [1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
    [1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
    [1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
    [1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
    [10, 1,  1,  1,  1,  1,  1,  1,  1, 10]
])

# Example use of find_top_k_peaks
coordinates_c = find_top_k_peaks(center, sigma=1, N=3)
coordinates_b = find_top_k_peaks(border, sigma=1, N=3)

print("Coordinates of top 3 peaks:")
print(coordinates_c)
print(coordinates_b)


Coordinates of top 3 peaks:
[[4 4]
 [4 5]
 [5 4]]
[[1 4]
 [1 5]
 [2 5]]


In [49]:
DPEAKS(center, border)

15

# Prueba de Import

In [15]:
from grav_lens import metrics

metrics.DICEE
metrics.DICE_binary_mask
metrics.DPEAKS
metrics.WMAPE

In [16]:
dir(metrics)

['DICEE',
 'DICE_binary_mask',
 'DPEAKS',
 'WMAPE',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'find_top_k_peaks',
 'scoring_challenge']